## Билет 1
### Решение систем линейных алгебраических уравнений методом Гаусса: расчетные формулы, подсчет числа действий.

In [1]:
#подключим библиотеку для работы с "матрицами"
import numpy as np

Метод Гаусса применяется для решения СЛАУ, в которых число уравнений равно числу неизвестных:
$$\sum_{j=1}^n{ a_{ij}x_j = f_i},\; i = 1,2,\ldots,n.$$
Или, в матричном виде:
$$Ax=f,\;A \in \mathbb{R}^{n \times n},\; det\,A \not = 0$$

Заметим, если $det \, A = 0 \Rightarrow \not\exists!$ решение. (вспоминаем первый семестр линала)

Метод Гаусса делится на два этапа:
1) прямой ход (forward elimination)
2) обратный ход (back substitution).

#### 1) Прямой ход.

Приведём матрицу $A$ к треугольному виду.
Пусть $a_{11} \not = 0$ (иначе поменяем местами строки с номерами $1$ и $i$, при котором $a_{i1} \not = 0$; поскольку $det\, A \not = 0$, такой номер $i$ заведомо найдется.
Разделим все члены первого уравнения на $a_{11}$:
$$c_{12}=\frac{a_{12}}{a_{11}},\;\ldots ,\; c_{1n}=\frac{a_{1n}}{a_{11}},\; y_1=\frac{f_1}{a_{11}}$$
и вычтем из каждого $i$-го уравнения системы ($i=2,\ldots,n$) преобразованное первое уравнение, умноженное на $a_{i1}$.

Преобразованная таким образом система примет вид:
$$ 
\begin{bmatrix}
    1 & c_{12}       & \ldots & c_{1n}       \\
    0 & a_{22}^{(1)} & \ldots & a_{2n}^{(1)} \\
    \vdots & \vdots  & \ddots & \vdots \\
    0 & a_{n2}^{(1)} & \ldots & a_{2n}^{(1)}
\end{bmatrix}
\begin{bmatrix}
    x_1 \\
    x_2 \\
    \vdots \\
    x_n 
\end{bmatrix} =
\begin{bmatrix}
    y_1 \\
    f_2^{(1)} \\
    \vdots \\
    f_n^{(1)}
\end{bmatrix} 
$$

Заметим, переменная $x_1$ исключилась из всех уравнений, кроме первого.
Проделаем аналогичные шаги для "укороченной" системы, содержащей (n-1) уравнений (из всех уравнений, кроме первого).

Продолжая процесс перейдем к системе:
$$ 
\begin{bmatrix}
    1 & c_{12} & c_{13} & \ldots & c_{1n} \\
    0 & 1      & c_{23} & \ldots & c_{2n} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    0 & 0      & 0      & \ldots & 1
\end{bmatrix}
\begin{bmatrix}
    x_1 \\
    x_2 \\
    \vdots \\
    x_n 
\end{bmatrix} =
\begin{bmatrix}
    y_1 \\
    y_2 \\
    \vdots \\
    y_3
\end{bmatrix}
$$

In [2]:
'''
В качестве параметра передаем матрицу [A|f]
'''
def forward_elimintaion(A):
    #ищем ненулевой ведущий элемент
    if A[0][0] == 0:
        for i in range(1, A.shape[0]):
            if A[i][0] != 0:
                break
        tmp = np.copy(A[0])
        A[0] = A[i]
        A[i] = tmp
    
    #делим первое уравнение и вычитаем его с коэффициентом из последующих уравнений
    A[0][0:] /= A[0][0]
    for i in range(1, A.shape[0]):
        A[i] -= A[i][0] * A[0]
    
    #повторяем те же действия для подматрицы
    if A.shape[0] > 1:
        forward_elimintaion(A[1:, 1:])

#### 2) Обратный ход.

Ищем корни СЛАУ.
Последовательно определяем неизвестные в обратном порядке:
$$
\begin{aligned}
x_n &= y_n \\
x_{n-1} &= y_{n-1} - c_{n-1,n}x_n \\
\cdots\\
x_1 &= y_1 - c_{12}x_2 - \ldots - c_{1n}x_n
\end{aligned}
$$

In [3]:
def back_substitution(A):
    ans = np.empty(A.shape[0])
    for i in range(A.shape[0] - 1, -1, -1):
        y_i = A[i][A.shape[1] - 1]
        C = A[i][i+1:A.shape[1]-1]
        ans[i] = y_i - (C*ans[i+1:]).sum()
    return ans

#### Пример:

In [4]:
A = np.array(
    [[1.2357, 2.1742, -5.4834],
     [3.4873, 6.1365, -4.7483],
     [6.0696, -6.2163, -4.6921]]
    )
f = np.array([-2.0735, 4.8755, -4.8388])
f.shape = (f.shape[0], 1)
A = np.hstack((A, f)) #прикрепляем к матрице A вектор-столбец f ([A|f])

In [5]:
forward_elimintaion(A)
print(A)

[[  1.00000000e+00   1.75948855e+00  -4.43748483e+00  -1.67799628e+00]
 [  0.00000000e+00   1.00000000e+00   1.68766938e+04   1.68776938e+04]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00   1.00000000e+00]]


In [6]:
print(back_substitution(A))

[ 1.  1.  1.]


Заметим, при $\bigl|c_{ij}\bigr| > 1 $ во время обратного хода умножение найденных с ошибками округления чисел $x_i$ на большие по модулю $c_{ij}$ приведет к увеличению этих ошибок.
Наоборот, если матрица $C$ оказалась такой, что:
$$ \left|c_{ij}\right| \leq 1,  $$
то роль ошибок округления нивелируется.

Такого условия можно добиться, выбирая на каждом шаге НАИБОЛЬШИЙ ПО МОДУЛЮ ненулевой ведущий элемент по столбцам.
(Тогда $\left|c_{ij}\right| = \displaystyle\left|\frac{a_{ij}}{\max_\limits{1 \leq j \leq n}{\left|a_{ij}\right|}}\right| \leq 1$)
Порядок корней в таком случае может измениться.

In [7]:
'''
Чтобы контролировать порядок корней, будем возвращать массив индексов
'''
def improved_forward_elimination(A):
    ind = np.arange(A.shape[0], dtype=int) #массив индексов корней
    
    for i in range(A.shape[0]):
        #ищем максимальный ведущий элемент по строке
        imax = np.argmax(np.abs(A[i, :A.shape[0]]))
        A[i][i], A[i][imax] = A[i][imax], A[i][i]
        ind[i], ind[imax] = ind[imax], ind[i]
        
        #делим уравнение и вычитаем его с коэффициентом из последующих уравнений
        A[i][i:] /= A[i][i]
        for j in range(i + 1, A.shape[0]):
            A[j] -= A[j][i] * A[i]
    
    return ind

#### Пример:

In [8]:
A = np.array(
    [[1.2357, 2.1742, -5.4834],
     [3.4873, 6.1365, -4.7483],
     [6.0696, -6.2163, -4.6921]]
    )
f = np.array([-2.0735, 4.8755, -4.8388])
f.shape = (f.shape[0], 1)
A = np.hstack((A, f)) #прикрепляем к матрице A вектор-столбец f ([A|f])

In [9]:
ind = improved_forward_elimination(A)
print(A)

[[ 1.         -0.39650582 -0.22535288  0.3781413 ]
 [ 0.          1.         -0.5269721   0.4730279 ]
 [ 0.          0.          1.          1.        ]]


In [10]:
print((back_substitution(A))[ind])

[ 1.  1.  1.]


### Подсчет числа действий
1) Прямой ход

Количество делений:
$$ Q_1 = n + (n-1) + \cdots + 1 = \frac{1}{2}n(n+1). $$
Количество сложений и умножений (не в сумме!):
$$ Q_2 = n(n-1) + (n-1)(n-2) + \cdots + 2 \cdot 1 = \frac{1}{3}n(n^2-1). $$

2) Обратный ход
Количество сложений и умножений (не в сумме!):
$$ Q_3 = 1 + 2 + \cdots + (n-1) = \frac{1}{2}n(n-1). $$

Таким образом общее число сложений и делений:
$$ Q_2 + Q_3 = \frac{1}{3}n(n-1)(n+\frac{5}{2}) = \frac{1}{3}n^3 + O(n^2) = O(n^3) $$

В принципе, норм